In [1]:
import sys
sys.path.append('..')
from lib import *
import pandas as pd
import numpy as np

forest = RNF(None, None, None, None, None, None, None)
forest.load_rnf('scen1.first600.64trees.10depth.42rs.20maxfeats.600maxin.pickle')

In [3]:
d = forest.get_feature_importances()

In [6]:
sorted([(d[key], key) for key in d.keys()])

[(0.060043646621125824, '8'),
 (0.06253897432944971, '58'),
 (0.07421186121917733, '61'),
 (0.08399591366586417, '56'),
 (0.11645793904596619, '99'),
 (0.13295330361560304, '92'),
 (0.1377902476118572, '83'),
 (0.14159195113471773, '77'),
 (0.14596645590053592, '84'),
 (0.15044322939632077, '81'),
 (0.17081511862781273, '94'),
 (0.17398035421943703, '25'),
 (0.17832673088637702, '24'),
 (0.18552360086929054, '95'),
 (0.18706799595226564, '65'),
 (0.1872717549337387, '53'),
 (0.1921479223986553, '54'),
 (0.19286316760329508, '19'),
 (0.20208647150273723, '12'),
 (0.20505745531040093, '91'),
 (0.2064867394242465, '87'),
 (0.20771156961537068, '68'),
 (0.22322340993205253, '90'),
 (0.22626661225852635, '43'),
 (0.22868880961994892, '74'),
 (0.23097151483232836, '27'),
 (0.2330531222144086, '75'),
 (0.23793504229771095, '52'),
 (0.24064214608606715, '14'),
 (0.2427048226267416, '88'),
 (0.24380835932599332, '67'),
 (0.25583249243858147, '57'),
 (0.2584895518335167, '17'),
 (0.2611981801256